In [1]:
import gym
import random, math
import numpy as np
import arcade
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from PIL import Image

        
# from LightEnvCopy import LightEnv
from LightEnvCopy import LightEnv

import gym.spaces
from gym.spaces import Discrete, Box

from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog

from collections import namedtuple

# Do the math to figure out our screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
SCREEN_TITLE = "Game 1: Let There Be Light!"

SPRITE_SCALING = 0.25

# How fast the camera pans to the player. 1.0 is instant.
CAMERA_SPEED = 0.1

PLAYER_MOVEMENT_SPEED = 7
BOMB_COUNT = 5
TORCH_COUNT = 1
PLAYING_FIELD_WIDTH = 800 #1600
PLAYING_FIELD_HEIGHT = 600 #1600
REWARD_COUNT = 1 #TF - Add in reward
END_GAME = False
torch_collected = False

#TF Start - Adding in actions for action conversion

# COnvenient data structure to hold information about actions
Action = namedtuple('Action', 'name index delta_i delta_j')

up = Action('up', 0, -1, 0)    
down = Action('down', 1, 1, 0)    
left = Action('left', 2, 0, -1)    
right = Action('right', 3, 0, 1)    

index_to_actions = {}
for action in [up, down, left, right]:
    index_to_actions[action.index] = action
# print(index_to_actions[0].name)
str_to_actions = {}
for action in [up, down, left, right]:
    str_to_actions[action.name] = action
#TF End - Adding in actions for action conversion


class LightEnvWrapper(gym.Env, LightEnv):
    """Class that wraps the Lights Environment to make it 
    compatible with RLLib."""

    metadata = {"render.modes": ["rgb_array", "state_pixels"]}
    
    def __init__(self, config: EnvContext):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
        self.counting = 0
#         game_size = config.get("size_env")

#         self.mygame = GUIWrapper(LightEnv)
        
        self.mygame = LightEnv
        #The action space is a choice of 9 actions: U/D/L/R/UR/DR/DL/UL/DO NOTHING. Not continuous
        #because speed of agent is fixed. Or potentially just 4: U/D/L/R.
        self.action_space = Discrete(4)
#         self.action_space = config.get("parrot_shriek_range", Discrete(4))
        #The observation space is a fixed image of the current game screen - fully observable.
        #Can set to a view just around the player using arcade.set_viewport.
        #Need obs space to be either 42,42,x or 84,84,x to be compatible with rllib.
        self.observation_space = Box(low=0, high=255, shape=(84,84, 4), dtype=np.uint8)
        
        self.counting = 0

    def reset(self):
        print("resetting in wrapper")
        self.render(drawing=True)
        #Resets the state of the environment for a new episode and an initial observation.
        obs_mygame = self.mygame.reset(self)
        
        #Open up the resetted image to verify working correctly.
        obs_mygame.show()
        
        #Convert observation to 84x84 resolution and np array for rllib.
        obs = self.convert_observations(obs_mygame)
        
#         print("resetted")
        return obs

    def step(self, action):
        self.counting += 1
        
        #Making sure an action is chosen, either: 0, 1, 2, 3.
        assert action in [0, 1, 2, 3] #0-up,1-down,2-left,3-right.
        
        #Convert the numeric action to a keyword: up, down, left, right.
        actions_myenv = index_to_actions[action].name #returns a word, one of: up/down/left/right
        print(f"action taken: {actions_myenv}")
        
        #Update the window with on_update()
        self.render(self)
        
        #Compute observation extracted from the window (800x600), with reward and done flag.
        obs, reward, done = self.mygame.step(self,actions_myenv)
        
        if self.counting % 33 == 0: #33 steps roughly equates to 1 second in game time
            print(f"total score is {self.score} at time: {self.mygame.time_taken_reported(self)}")
        
        #Convert observation to 84x84 resolution and np array for rllib.
        obs_mygame = self.convert_observations(obs)
        
        #If the reward has been obtained, reset the environment and start again
        if done == True:
            print(f"done is {done}, resetting environment in wrapper.")
            self.reset()
        
        return obs_mygame, reward, done, {}

    def seed(self, seed=None):
        random.seed(seed)

    def convert_observations(self, obs_mygame): #resizing and converting to array for rllib processing
        # We normalize and concatenate observations
        obs = obs_mygame
        obs_resized = obs.resize((84,84))
        obsarray = np.array(obs_resized)
        return obsarray
    
    def render(self, mode='state_pixels', drawing=False):
        self.mygame.on_update(self, 1/60)
        self.mygame.on_draw(self)
        test = self.mygame.time_taken_reported(self)
        

### Now run the rllib script to train the agent

In [2]:
# !pip install ray==1.11.0
import gym
import ray.rllib.agents.ppo.ppo as ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air
from ray import tune
config = PPOConfig()


# import ray.rllib.agents.dqn as dqn
from ray.tune.logger import pretty_print



from ray.rllib.algorithms.ppo import PPOConfig
config = PPOConfig().training(gamma=0.9, lr=0.1, kl_coeff=0.9)\
            .resources(num_gpus=0)\
            .rollouts(num_envs_per_worker=1, num_rollout_workers=1)\
            .policies(framework = 'torch')
config.normalize_actions=True
config.env=LightEnvWrapper
config.action_space=Discrete(4)
config.observation_space=Box(low=0, high=255, shape=(84,84, 4), dtype=np.uint8)
config.clip_actions=False
# config.env_config={"parrot_shriek_range": Discrete(5)}
print(config.to_dict())
# Build a Algorithm object from the config and run 1 training iteration.
# trainer = config.build(env=LightEnvWrapper)
# trainer.train()
trainer = ppo.PPOTrainer(config=config)
# trainer.train()

avg_rewards = []
num_iterations = []
time_spent = []
for episode in range(1000):
    print("Starting episode ", episode)
    # Perform one iteration of training the policy with PPO
    result = trainer.train()
    #print(pretty_print(result))
    print("episode reward mean: ", result['episode_reward_mean'])
    avg_rewards.append(result['episode_reward_mean'])
    num_iterations.append(episode)
    if episode % 10 == 0:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)
    print("End of episode ", episode)



    

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': <class '__main__.LightEnvWrapper'>, 'env_config': {}, 'observation_space': Box(0, 255, (84, 84, 4), uint8), 'action_space': Discrete(4), 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': False, 'num_workers': 1, 'num_envs_per_worker': 1, 'sample_collector': <class 'ray.rllib.evaluation.collectors.simple_

C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\ray\_private\ray_option_utils.py:266: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.0/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
2022-09-12 15:31:13,023	INFO worker.py:1518 -- Started a local Ray instance.
(pid=5124) Windows fatal exception: code 0xc0000139
(pid=5124) 
(RolloutWorker pid=5124) 2022-09-12 15:31:25,781	WARNING env.py:142 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


(RolloutWorker pid=5124) resetting in wrapper
(RolloutWorker pid=5124) resetting
(RolloutWorker pid=5124) action taken: down


2022-09-12 15:31:32,477	INFO trainable.py:160 -- Trainable.setup took 22.181 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-09-12 15:31:32,479	WARNING util.py:65 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=5124) resetting in wrapper
(RolloutWorker pid=5124) resetting
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: 

...

(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) total score is 0 at time: 59
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=512

(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action 

(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) total score is 0 at time: 64
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=512

(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) total score is 0 at time: 67
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left
(RolloutWorker pid=5124) action taken: left


{'custom_metrics': {},
 'episode_media': {},
 'num_recreated_workers': 0,
 'info': {'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.8999999761581421,
     'cur_lr': 0.10000000149011612,
     'total_loss': 9.986912,
     'policy_loss': -0.01303174,
     'vf_loss': 9.999944,
     'vf_explained_var': -0.9989254,
     'kl': 8.917524e-16,
     'entropy': 1.3373474e-12,
     'entropy_coeff': 0.0,
     'model': {}},
    'custom_metrics': {},
    'num_agent_steps_trained': 128.0}},
  'num_env_steps_sampled': 4000,
  'num_env_steps_trained': 4000,
  'num_agent_steps_sampled': 4000,
  'num_agent_steps_trained': 4000},
 'sampler_results': {'episode_reward_max': nan,
  'episode_reward_min': nan,
  'episode_reward_mean': nan,
  'episode_len_mean': nan,
  'episode_media': {},
  'episodes_this_iter': 0,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [], 'episode_lengths': []},
  'sampler_p